In [1]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-clean-mooccubex/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-clean-mooccubex/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-clean-mooccubex/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-clean-mooccubex/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-clean-mooccubex/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validation = {
    "week1": [
        "/kaggle/input/new-clean-mooccubex/clean_week1/val/val_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-clean-mooccubex/clean_week2/val/val_week1_2.csv"
    ],
    "week3": [
        "/kaggle/input/new-clean-mooccubex/clean_week3/val/val_week1_2_3.csv"
    ],
    "week4": [
        "/kaggle/input/new-clean-mooccubex/clean_week4/val/val_week1_2_3_4.csv"
    ]
}

file_test = {
    "week1": [
        "/kaggle/input/new-clean-mooccubex/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-clean-mooccubex/clean_week1/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-clean-mooccubex/clean_week1/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-clean-mooccubex/clean_week1/test/test_week4.csv"
    ]
}

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import time
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support
from lightgbm import LGBMClassifier
import xgboost as xgb
import lightgbm as lgb
import numpy as np

## Tìm siêu tham số tốt nhất cho từng tuần

In [3]:
# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
       # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    model = LGBMClassifier(random_state=42)
       # # # Khởi tạo RandomSearch tuner
    # param_grid = {
    #     'n_estimators': [100, 200, 300],
    #     'max_depth': [3, 5, 7],
    #     'learning_rate': [0.05, 0.1, 0.2],
    #     'num_leaves': [20, 30, 40],
    #     'min_child_samples': [20, 30],
    #     'subsample': [0.8, 1.0],
    #     'colsample_bytree': [0.8, 1.0],
    #     'reg_alpha': [0, 0.1],
    #     'reg_lambda': [0, 0.1, 0.5]
    # }
    param_grid = {
        'n_estimators': [100, 200],               # Giảm số lượng giá trị
        'learning_rate': [0.05, 0.1],            # Tập trung vào các giá trị phổ biến
        'max_depth': [3, 5],                     # Giới hạn độ sâu
        'num_leaves': [20, 31],                  # Giảm số giá trị thử nghiệm
        'subsample': [0.8],                      # Cố định tỷ lệ lấy mẫu
        'colsample_bytree': [0.8],               # Cố định tỷ lệ cột
        'reg_alpha': [0, 0.1],                   # Giới hạn các giá trị regularization
        'reg_lambda': [0, 0.1]
    }
        
    # Khởi tạo RandomSearch tuner
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, scoring='accuracy',
                              verbose=2, n_jobs=-1)
        
    
   # Huấn luyện mô hình với các siêu tham số tốt nhất
    grid_search.fit(X_train_res, y_train_res)

    # Đánh giá mô hình
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_val)
    # Trả về kết quả tối ưu cho tuần
    best_params = grid_search.best_params_
    return best_params

In [4]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    "week1": "/kaggle/input/new-clean-mooccubex/clean_week1/train/clean_data_week1.csv",
    "week2": "/kaggle/input/new-clean-mooccubex/clean_week2/train/clean_data_week2.csv",
    "week3": "/kaggle/input/new-clean-mooccubex/clean_week3/train/clean_data_week3.csv",
    "week4": "/kaggle/input/new-clean-mooccubex/clean_week4/train/clean_data_week4.csv"
}

file_validation = {
    "week1": "/kaggle/input/new-clean-mooccubex/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-clean-mooccubex/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-clean-mooccubex/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-clean-mooccubex/clean_week4/val/val_week1_2_3_4.csv"
}

## Tham số tốt nhất cho từng tuần

In [5]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name, param_value in best_params_week1.items():
       print(f"{param_name}: {param_value}")

print("\nBest Parameters for Week 2:")
for param_name, param_value in best_params_week2.items():
       print(f"{param_name}: {param_value}")

print("\nBest Parameters for Week 3:")
for param_name, param_value in best_params_week3.items():
       print(f"{param_name}: {param_value}")

print("\nBest Parameters for Week 4:")
for param_name, param_value in best_params_week4.items():
       print(f"{param_name}: {param_value}")

Fitting 10 folds for each of 64 candidates, totalling 640 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002957 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2464
[LightGBM] [Info] Number of data points in the train set: 40085, number of used features: 10
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

In [6]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

## LightGBM với các tham sốt nhất cho mỗi tuần

In [7]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week]
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = train_data['classification_encoded']
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = test_data['classification_encoded']


        # Xây dựng mô hình với tham số tốt nhất
        model = lgb.LGBMClassifier(**params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        accuracy = accuracy_score(y_test, y_pred)
        week_results["accuracy_per_fold"].append(accuracy)
        # Dự đoán

        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
        conf_matrix = confusion_matrix(y_test, y_pred)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix|
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'num_leaves': 31, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.8}
Fold 1: Using file /kaggle/input/new-clean-mooccubex/clean_week1/train/5-folds/data_part_1.csv as test set
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1565
[LightGBM] [Info] Number of data points in the train set: 10489, number of used features: 10
[LightGBM] [Info] Start training from score -1.474858
[LightGBM] [Info] Start training from score -3.400149
[LightGBM] [Info] Start training from score -2.768877
[LightGBM] [Info] Start training from score -2.753794
[LightGBM] [Info] Start training from score -0.492000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

## Kết quả 5-folds

In [8]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 0.5614 seconds
Average Test Time: 0.0225 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.999003        0.999667          0.999334
1      1           1.000000        1.000000          1.000000
2      2           1.000000        0.998788          0.999392
3      3           1.000000        0.995210          0.997594
4      4           0.999501        0.999875          0.999688

Average Confusion Matrix:
       0     1      2      3       4
0  599.8   0.0    0.0    0.0     0.2
1    0.0  87.6    0.0    0.0     0.0
2    0.2   0.0  164.2    0.0     0.0
3    0.2   0.0    0.0  166.2     0.6
4    0.2   0.0    0.0    0.0  1603.2

=== Results for week2 ===
Average Train Time: 1.0968 seconds
Average Test Time: 0.0789 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.999667  

## Test trên tập test

In [9]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[week]
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-clean-mooccubex/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-clean-mooccubex/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Xây dựng mô hình với tham số tốt nhất
    model = lgb.LGBMClassifier(**params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test), columns=np.unique(y_test)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [10]:
process_week(1, best_params, results)


=== Processing Week 1 ===
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1599
[LightGBM] [Info] Number of data points in the train set: 13112, number of used features: 10
[LightGBM] [Info] Start training from score -1.474916
[LightGBM] [Info] Start training from score -3.399064
[LightGBM] [Info] Start training from score -2.769543
[LightGBM] [Info] Start training from score -2.753851
[LightGBM] [Info] Start training from score -0.491964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

In [11]:
process_week(2, best_params, results)


=== Processing Week 2 ===
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001327 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2621
[LightGBM] [Info] Number of data points in the train set: 13112, number of used features: 17
[LightGBM] [Info] Start training from score -1.474916
[LightGBM] [Info] Start training from score -3.399064
[LightGBM] [Info] Start training from score -2.769543
[LightGBM] [Info] Start training from score -2.753851
[LightGBM] [Info] Start training from score -0.491964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

In [12]:
process_week(3, best_params, results)


=== Processing Week 3 ===
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000603 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3514
[LightGBM] [Info] Number of data points in the train set: 13112, number of used features: 24
[LightGBM] [Info] Start training from score -1.474916
[LightGBM] [Info] Start training from score -3.399064
[LightGBM] [Info] Start training from score -2.769543
[LightGBM] [Info] Start training from score -2.753851
[LightGBM] [Info] Start training from score -0.491964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

In [13]:
process_week(4, best_params, results)


=== Processing Week 4 ===
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4347
[LightGBM] [Info] Number of data points in the train set: 13112, number of used features: 31
[LightGBM] [Info] Start training from score -1.474916
[LightGBM] [Info] Start training from score -3.399064
[LightGBM] [Info] Start training from score -2.769543
[LightGBM] [Info] Start training from score -2.753851
[LightGBM] [Info] Start training from score -0.491964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

In [14]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 0.37 seconds
  Test Time: 0.01 seconds
  Precision: [1.         1.         1.         1.         0.99900398]
  Recall: [1.         1.         1.         0.99047619 1.        ]
  F1-Score: [1.         1.         1.         0.99521531 0.99950174]
  Confusion Matrix:
[[ 375    0    0    0    0]
 [   0   54    0    0    0]
 [   0    0  103    0    0]
 [   0    0    0  104    1]
 [   0    0    0    0 1003]]
Week 2:
  Train Time: 0.46 seconds
  Test Time: 0.01 seconds
  Precision: [1. 1. 1. 1. 1.]
  Recall: [1. 1. 1. 1. 1.]
  F1-Score: [1. 1. 1. 1. 1.]
  Confusion Matrix:
[[ 375    0    0    0    0]
 [   0   54    0    0    0]
 [   0    0  103    0    0]
 [   0    0    0  105    0]
 [   0    0    0    0 1003]]
Week 3:
  Train Time: 0.49 seconds
  Test Time: 0.01 seconds
  Precision: [1. 1. 1. 1. 1.]
  Recall: [1. 1. 1. 1. 1.]
  F1-Score: [1. 1. 1. 1. 1.]
  Confusion Matrix:
[[ 375    0    0    0    0]
 [   0   54    0    0    0]
 [